In [1]:
import pandas as pd
from datetime import datetime

In [2]:
cols = ['ts_code', 'start_date', 'is_release', 'release_date']
data = pd.read_csv('../data/pledge_llf.csv', usecols=cols, parse_dates=['start_date', 'release_date'])

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99284 entries, 0 to 99283
Data columns (total 4 columns):
ts_code         99284 non-null object
start_date      97521 non-null object
is_release      93436 non-null float64
release_date    53040 non-null object
dtypes: float64(1), object(3)
memory usage: 3.0+ MB


In [4]:
data = data.dropna(subset=['start_date'])

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97521 entries, 0 to 99283
Data columns (total 4 columns):
ts_code         97521 non-null object
start_date      97521 non-null object
is_release      92392 non-null float64
release_date    51335 non-null object
dtypes: float64(1), object(3)
memory usage: 3.7+ MB


In [6]:
data.head()

,ts_code,start_date,is_release,release_date
0,000002.SZ,20181225,1.0,20190826
1,000002.SZ,20190827,0.0,NaN
2,000002.SZ,20181218,1.0,20190722
3,000002.SZ,20190611,0.0,NaN
4,000002.SZ,20180926,1.0,20190605


In [7]:
data['len'] = data.apply(lambda x: len(x.start_date), axis=1)

In [8]:
data = data[data['len'] == 8]

In [9]:
data['start_date'] = pd.to_datetime(data['start_date'])

In [10]:
data.head()

,ts_code,start_date,is_release,release_date,len
0,000002.SZ,2018-12-25,1.0,20190826,8
1,000002.SZ,2019-08-27,0.0,NaN,8
2,000002.SZ,2018-12-18,1.0,20190722,8
3,000002.SZ,2019-06-11,0.0,NaN,8
4,000002.SZ,2018-09-26,1.0,20190605,8


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97452 entries, 0 to 99283
Data columns (total 5 columns):
ts_code         97452 non-null object
start_date      97452 non-null datetime64[ns]
is_release      92347 non-null float64
release_date    51281 non-null object
len             97452 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 4.5+ MB


In [12]:
data = data[(data['start_date'] > '2016-01-01') & (data['start_date'] < '2019-08-01')]

In [13]:
data = data.drop_duplicates()

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54871 entries, 0 to 99283
Data columns (total 5 columns):
ts_code         54871 non-null object
start_date      54871 non-null datetime64[ns]
is_release      54871 non-null float64
release_date    27640 non-null object
len             54871 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 2.5+ MB


In [15]:
data = data.drop(['len'], axis=1)

In [16]:
data.to_csv('../data/pledge_llf_clean.csv')

In [17]:
data.head()

,ts_code,start_date,is_release,release_date
0,000002.SZ,2018-12-25,1.0,20190826
2,000002.SZ,2018-12-18,1.0,20190722
3,000002.SZ,2019-06-11,0.0,NaN
4,000002.SZ,2018-09-26,1.0,20190605
5,000002.SZ,2018-09-28,1.0,20190605


In [18]:
price_dict = {}
for code in set(data['ts_code']):
    price_df = pd.read_csv('../data/price/{}.csv'.format(code), index_col=0)
    price_df.index = pd.to_datetime(price_df.index)
    price_dict[code] = price_df

In [19]:
len(price_dict)

2413

In [20]:
price_dict['000002.SZ'].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 761 entries, 2016-07-04 to 2019-08-21
Data columns (total 9 columns):
open         761 non-null float64
high         761 non-null float64
low          761 non-null float64
close        761 non-null float64
pre_close    761 non-null float64
change       761 non-null float64
pct_chg      761 non-null float64
vol          761 non-null float64
amount       761 non-null float64
dtypes: float64(9)
memory usage: 59.5 KB


In [21]:
price_dict['000002.SZ'].head()

,open,high,low,close,pre_close,change,pct_chg,vol,amount
trade_date,,,,,,,,,
2016-07-04,18.8836,18.8836,18.8836,18.8836,20.9790,-2.0954,-9.9881,42663.00,9.381594e+04
2016-07-05,16.9944,16.9944,16.9944,16.9944,18.8836,-1.8892,-10.0044,1990576.00,3.939350e+06
2016-07-06,16.4019,17.8446,16.4019,17.0030,16.9944,0.0086,0.0506,10283727.12,2.010649e+07
2016-07-07,16.4019,16.5049,16.0670,16.1614,17.0030,-0.8416,-4.9497,4300225.52,8.133978e+06
2016-07-08,15.9725,16.2988,15.6376,16.1013,16.1614,-0.0601,-0.3719,3124436.98,5.799513e+06


In [56]:
price_dict['000002.SZ'].iloc[1].name

Timestamp('2016-07-05 00:00:00')

In [22]:
data['has_price'] = data.apply(lambda x: x.start_date in price_dict[x.ts_code].index, axis=1)

In [23]:
data[data['has_price']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51087 entries, 0 to 99283
Data columns (total 5 columns):
ts_code         51087 non-null object
start_date      51087 non-null datetime64[ns]
is_release      51087 non-null float64
release_date    25618 non-null object
has_price       51087 non-null bool
dtypes: bool(1), datetime64[ns](1), float64(1), object(2)
memory usage: 2.0+ MB


In [24]:
data = data[data['has_price']]

In [25]:
data['pledge_price'] = data.apply(lambda x: price_dict[x.ts_code].loc[x.start_date]['low'], axis=1)

In [26]:
data = data.drop(['has_price'], axis=1)

In [27]:
data.head()

,ts_code,start_date,is_release,release_date,pledge_price
0,000002.SZ,2018-12-25,1.0,20190826,22.4751
2,000002.SZ,2018-12-18,1.0,20190722,24.3593
3,000002.SZ,2019-06-11,0.0,NaN,26.7721
4,000002.SZ,2018-09-26,1.0,20190605,22.9654
5,000002.SZ,2018-09-28,1.0,20190605,23.0231


In [28]:
data['forcast_close_line'] = data.apply(lambda x: x.pledge_price * 0.7, axis=1)

In [29]:
data.head()

,ts_code,start_date,is_release,release_date,pledge_price,forcast_close_line
0,000002.SZ,2018-12-25,1.0,20190826,22.4751,15.73257
2,000002.SZ,2018-12-18,1.0,20190722,24.3593,17.05151
3,000002.SZ,2019-06-11,0.0,NaN,26.7721,18.74047
4,000002.SZ,2018-09-26,1.0,20190605,22.9654,16.07578
5,000002.SZ,2018-09-28,1.0,20190605,23.0231,16.11617


In [30]:
data.groupby(['is_release']).count()

,ts_code,start_date,release_date,pledge_price,forcast_close_line
is_release,,,,,
0.0,25462,25462,0,25462,25462
1.0,25625,25625,25618,25625,25625


In [36]:
def filter_release_date(x):
    # 还未解押
    if int(x.is_release) == 0:
        return True
    # 已解押但没有时间数据
    if x.release_date is None:
        return False
    # 已解押但时间数据非法
    if len(str(x.release_date)) != 8:
        return False
    return True
        

In [37]:
data['is_valid_release_date'] = data.apply(filter_release_date, axis=1)

In [38]:
data[data['is_valid_release_date']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51080 entries, 0 to 99283
Data columns (total 7 columns):
ts_code                  51080 non-null object
start_date               51080 non-null datetime64[ns]
is_release               51080 non-null float64
release_date             25618 non-null object
pledge_price             51080 non-null float64
forcast_close_line       51080 non-null float64
is_valid_release_date    51080 non-null bool
dtypes: bool(1), datetime64[ns](1), float64(3), object(2)
memory usage: 2.8+ MB


In [39]:
data = data[data['is_valid_release_date']]

In [40]:
data.head()

,ts_code,start_date,is_release,release_date,pledge_price,forcast_close_line,is_valid_release_date
0,000002.SZ,2018-12-25,1.0,20190826,22.4751,15.73257,True
2,000002.SZ,2018-12-18,1.0,20190722,24.3593,17.05151,True
3,000002.SZ,2019-06-11,0.0,NaN,26.7721,18.74047,True
4,000002.SZ,2018-09-26,1.0,20190605,22.9654,16.07578,True
5,000002.SZ,2018-09-28,1.0,20190605,23.0231,16.11617,True


In [41]:
data = data.drop(['is_valid_release_date'], axis=1)

In [42]:
data['release_date'] = pd.to_datetime(data['release_date'])

In [43]:
data.head()

,ts_code,start_date,is_release,release_date,pledge_price,forcast_close_line
0,000002.SZ,2018-12-25,1.0,2019-08-26,22.4751,15.73257
2,000002.SZ,2018-12-18,1.0,2019-07-22,24.3593,17.05151
3,000002.SZ,2019-06-11,0.0,NaT,26.7721,18.74047
4,000002.SZ,2018-09-26,1.0,2019-06-05,22.9654,16.07578
5,000002.SZ,2018-09-28,1.0,2019-06-05,23.0231,16.11617


In [58]:
def is_close(x):
    p_df = price_dict[x.ts_code]
    if int(x.is_release) == 1:
        p_df = p_df[x.start_date: x.release_date]
    else:
        p_df = p_df[x.start_date:]
    smaller_than_close = p_df[p_df['close'] < x.forcast_close_line]
    if len(smaller_than_close) > 0:
        return True, smaller_than_close.iloc[0].name
    else:
        return False, None

In [63]:
# data['is_reach_close_line'], data['close_date'] = data.apply(is_close, axis=1)
two_df = pd.DataFrame(data.apply(lambda x: is_close(x), axis=1).to_list(), 
             columns=['is_reach_close_line', 'close_date'],
            index=data.index)

In [65]:
data = pd.concat([data, two_df], axis=1)

In [69]:
data.groupby('is_reach_close_line').count()

,ts_code,start_date,is_release,release_date,pledge_price,forcast_close_line,close_date
is_reach_close_line,,,,,,,
False,27293,27293,27293,16054,27293,27293,0
True,23787,23787,23787,9564,23787,23787,23787


In [70]:
data = data.drop(['is_release'], axis=1)

In [71]:
data.head()

,ts_code,start_date,release_date,pledge_price,forcast_close_line,is_reach_close_line,close_date
0,000002.SZ,2018-12-25,2019-08-26,22.4751,15.73257,False,NaT
2,000002.SZ,2018-12-18,2019-07-22,24.3593,17.05151,False,NaT
3,000002.SZ,2019-06-11,NaT,26.7721,18.74047,False,NaT
4,000002.SZ,2018-09-26,2019-06-05,22.9654,16.07578,False,NaT
5,000002.SZ,2018-09-28,2019-06-05,23.0231,16.11617,False,NaT


In [72]:
data.to_csv('../data/pledge_full_clean.csv', index=False)

In [75]:
pledge_positive = data[data['is_reach_close_line']]
pledge_neg = data[data['is_reach_close_line'] == False]

In [76]:
pledge_positive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23787 entries, 13 to 99276
Data columns (total 7 columns):
ts_code                23787 non-null object
start_date             23787 non-null datetime64[ns]
release_date           9564 non-null datetime64[ns]
pledge_price           23787 non-null float64
forcast_close_line     23787 non-null float64
is_reach_close_line    23787 non-null bool
close_date             23787 non-null datetime64[ns]
dtypes: bool(1), datetime64[ns](3), float64(2), object(1)
memory usage: 1.3+ MB


In [77]:
pledge_positive.head()

,ts_code,start_date,release_date,pledge_price,forcast_close_line,is_reach_close_line,close_date
13,000002.SZ,2017-12-01,2019-01-17,28.2164,19.75148,True,2018-08-03
22,000002.SZ,2018-01-30,2018-12-24,33.5544,23.48808,True,2018-05-30
27,000002.SZ,2017-12-01,2018-11-30,28.2164,19.75148,True,2018-08-03
28,000002.SZ,2017-12-01,NaT,28.2164,19.75148,True,2018-08-03
30,000002.SZ,2017-12-01,2018-11-28,28.2164,19.75148,True,2018-08-03


In [78]:
pledge_positive = pledge_positive.drop(['release_date'], axis=1)

In [80]:
pledge_positive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23787 entries, 13 to 99276
Data columns (total 6 columns):
ts_code                23787 non-null object
start_date             23787 non-null datetime64[ns]
pledge_price           23787 non-null float64
forcast_close_line     23787 non-null float64
is_reach_close_line    23787 non-null bool
close_date             23787 non-null datetime64[ns]
dtypes: bool(1), datetime64[ns](2), float64(2), object(1)
memory usage: 1.1+ MB


In [79]:
pledge_neg.head()

,ts_code,start_date,release_date,pledge_price,forcast_close_line,is_reach_close_line,close_date
0,000002.SZ,2018-12-25,2019-08-26,22.4751,15.73257,False,NaT
2,000002.SZ,2018-12-18,2019-07-22,24.3593,17.05151,False,NaT
3,000002.SZ,2019-06-11,NaT,26.7721,18.74047,False,NaT
4,000002.SZ,2018-09-26,2019-06-05,22.9654,16.07578,False,NaT
5,000002.SZ,2018-09-28,2019-06-05,23.0231,16.11617,False,NaT


In [81]:
pledge_neg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27293 entries, 0 to 99283
Data columns (total 7 columns):
ts_code                27293 non-null object
start_date             27293 non-null datetime64[ns]
release_date           16054 non-null datetime64[ns]
pledge_price           27293 non-null float64
forcast_close_line     27293 non-null float64
is_reach_close_line    27293 non-null bool
close_date             0 non-null datetime64[ns]
dtypes: bool(1), datetime64[ns](3), float64(2), object(1)
memory usage: 1.5+ MB


In [82]:
pledge_neg = pledge_neg.drop(['close_date'], axis=1)

In [83]:
pledge_neg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27293 entries, 0 to 99283
Data columns (total 6 columns):
ts_code                27293 non-null object
start_date             27293 non-null datetime64[ns]
release_date           16054 non-null datetime64[ns]
pledge_price           27293 non-null float64
forcast_close_line     27293 non-null float64
is_reach_close_line    27293 non-null bool
dtypes: bool(1), datetime64[ns](2), float64(2), object(1)
memory usage: 1.3+ MB


In [84]:
pledge_positive.to_csv('../data/pledge/pledge_pos.csv', index=False)
pledge_neg.to_csv('../data/pledge/pledge_neg.csv', index=False)